In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

In [ ]:
if endpoint == "" or api_key == "" or deployment == "":
    print("Bitte trage die API Daten in .env Datei ein. ")

In [ ]:
client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2024-08-01-preview"
)

## Senden von Nachrichten

In [ ]:
message_text = [
    {"role":"system","content":"Du bist ein AI Assistent der Menschen hilft Antworten zu finden."},
    {"role":"user","content":"Was macht die Impact Labs GmbH aus Dresden?"},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text
)

print(completion.choices[0].message.content)

## Streaming von Antworten

In [ ]:
completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,
  stream=True,
)

for element in completion: 
    if len(element.choices) > 0:    
        print(element.choices[0].delta.content, end="", flush=True)

## Structured Outputs - ein JSON als Antwort

In [ ]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model=deployment,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Tina und Toni treffen sich am 12.12.2026 zum einer Tombola."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

print(event)